In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd
import itertools

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM_adv"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/22_01_09_fedavg_n80_benign/'
args_.validation = False
args_.num_user = 40

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 232.07it/s]


===> Initializing clients..


 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                           | 60/80 [00:40<00:13,  1.50it/s]


KeyboardInterrupt: 

In [ ]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

In [ ]:
# Import Model Weights
setting = 'FedAvg'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

adv_mode = False    

num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

    if adv_mode:
        args_.save_path ='weights/cifar100/feddef_l2/local/'
        aggregator.load_state(args_.save_path)

        model_weights = []
        weights = np.load('weights/cifar100/feddef_l2/local/train_client_weights.npy')
    else:
        args_.save_path ='weights/cifar100/first_test/local/'
        aggregator.load_state(args_.save_path)

        model_weights = []
        weights = np.load('weights/cifar100/first_test/local/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    
    if adv_mode:
        args_.save_path = 'weights/cifar100/feddef_l2/fedavg/'
    else:
        args_.save_path = 'weights/cifar100/first_test/fedavg/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]
    
    if adv_mode:
        weights = np.load('weights/cifar100/feddef_l2/fedavg/train_client_weights.npy')
    else:
        weights = np.load('weights/cifar100/first_test/fedavg/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    if adv_mode:
        args_.save_path = 'weights/neurips/cifar/krum/just_attack/adv/'
    else:
        args_.save_path = 'weights/neurips/cifar/krum/just_attack/benign/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    if adv_mode:
        weights = np.load('weights/neurips/cifar/krum/just_attack/adv/train_client_weights.npy')
    else:
        weights = np.load('weights/neurips/cifar/krum/just_attack/benign/train_client_weights.npy')

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [ ]:
for learner_id, learner in enumerate(aggregator.global_learners_ensemble):
    print(learner_id)
    print(learner)

In [ ]:
learners = [client.learners_ensemble[learner_id] for client in aggregator.clients]
len(learners)

In [ ]:
target_learner = learner
learners = learners

target_state_dict = target_learner.model.state_dict(keep_vars=True)
for key in target_state_dict:
    print(key)
    if target_state_dict[key].data.dtype == torch.float32:
        
        distance_matrix = np.zeros([len(learners),len(learners)])
        state_dict_vals_log = []
        
        for learner_id, learner in enumerate(learners):
            state_dict_vals_log += [learner.model.state_dict(keep_vars=True)[key].cpu().detach().numpy()]
            
        for i1,i2 in itertools.product(range(len(learners)),range(len(learners))):
            if i1 != i2 and distance_matrix[i1, i2] == 0:
                c = state_dict_vals_log[i1]- state_dict_vals_log[i2]
                distance_matrix[i1, i2] = np.linalg.norm(c)
                distance_matrix[i2, i1] = np.linalg.norm(c)
    
        # from distance matrix calculate mean for n-f-2, n=num learners, f = num_sybl, 2
        krum_vector = np.zeros(len(learners))
        # value of k
        k = len(learners) - f - 2  

        # using np.argpartition()
        for i in range(len(learners)):
            result = np.argpartition(distance_matrix[i], k)
            krum_vector[i] = np.sum(distance_matrix[i][result[:k]])
            
        # Update weight using krum 
        min_idx = np.argmin(krum_vector)
        target_state_dict[key].data = learners[min_idx].model.state_dict(keep_vars=True)[key].data.clone()
        
    else:
        # tracked batches
        target_state_dict[key].data.fill_(0)
        for learner_id, learner in enumerate(learners):
            state_dict = learner.model.state_dict()
            target_state_dict[key].data += state_dict[key].data.clone()

In [ ]:
arr =np.random.uniform(0,1,[len(learners),len(learners)])

krum_vector = np.zeros(len(learners))

# value of k
n = 40
f = 10
k = n-f-2  
    
# using np.argpartition()
for i in range(len(learners)):
    result = np.argpartition(arr[i], k)
    krum_vector[i] = np.sum(arr[i][result[:k]])


In [ ]:
krum_vector

In [91]:
learners[0].model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [75]:
learners[0]